In [1]:
import os
# Move to Thesis directory (two levels up)
os.chdir(os.path.abspath(os.path.join("..", "..")))

# Move to model/src if it exists
model_dir = os.path.join(os.getcwd(), "model", "src")
if os.path.exists(model_dir):
    os.chdir(model_dir)

print("Current Directory:", os.getcwd())

Current Directory: c:\Users\Utilizador\OneDrive - NOVAIMS\Desktop\github\Thesis\model\src


In [2]:
import os
from langchain_core.documents import Document
import pickle
import re
import torch

In [5]:
# Folder where your pickle files are stored
pickle_folder = r"../../data/Preprocessing_text/bachelors_data"

# List to store all LangChain Documents
all_documents = []

# Loop through all files in the folder
for filename in os.listdir(pickle_folder):
    if filename.endswith(".pkl"):
        file_path = os.path.join(pickle_folder, filename)
        with open(file_path, "rb") as f:
            docs_dict = pickle.load(f)
        
        # Convert each key-value pair into a Document object
        documents = [
            Document(page_content=text, metadata={"source": key})
            for key, text in docs_dict.items()
        ]
        
        all_documents.extend(documents)

print(f"Total documents loaded: {len(all_documents)}")
print("Sample document metadata:", all_documents[0].metadata)

Total documents loaded: 9
Sample document metadata: {'source': 'data-science_main_course_extracted_text.txt'}


In [6]:
# Check how many documents were loaded
print(f"Total documents loaded: {len(all_documents)}")

# Print a sample document (first one)
print("\nSample Document Object:")
print(all_documents[0])

# Print just the metadata and a preview of content
print("\nMetadata and preview:")
print("Metadata:", all_documents[0].metadata)
print("Content preview:", all_documents[0].page_content[:300])  # first 300 chars

Total documents loaded: 9

Sample Document Object:
page_content='Text from https://www.novaims.unl.pt/en///bachelor-s-degrees/data-science/: Data Science Degree in Data Science en Bachelor's Degrees Data Science In the Bachelor´s Degree in Data Science, students learn the most modern techniques of artificial intelligence and machine learning to analyze large volumes of data (Big Data). They will become true data scientists - considered the sexiest profession of the 21 st century by the Harvard Business Review. The main objective of this course is to train future professionals capable of understanding, developing and using models, algorithms and the most advanced techniques in data science, to analyze and extract knowledge from Big Data. The 3 rd phase of applications under the International Student Statute for the 2025/26 academic year are open from February 26 th to March 27 th , 2025. Duration 3 years (6 semesters) Timetable Daytime Start September 2025 Career Opportunities The Bache

In [7]:
all_documents

[Document(metadata={'source': 'data-science_main_course_extracted_text.txt'}, page_content="Text from https://www.novaims.unl.pt/en///bachelor-s-degrees/data-science/: Data Science Degree in Data Science en Bachelor's Degrees Data Science In the Bachelor´s Degree in Data Science, students learn the most modern techniques of artificial intelligence and machine learning to analyze large volumes of data (Big Data). They will become true data scientists - considered the sexiest profession of the 21 st century by the Harvard Business Review. The main objective of this course is to train future professionals capable of understanding, developing and using models, algorithms and the most advanced techniques in data science, to analyze and extract knowledge from Big Data. The 3 rd phase of applications under the International Student Statute for the 2025/26 academic year are open from February 26 th to March 27 th , 2025. Duration 3 years (6 semesters) Timetable Daytime Start September 2025 Car

In [8]:
def split_teaching_staff_grouped(text, metadata=None, max_token_size=200):
    # Regex pattern to match a block of "Name\nTitle\nEmail"
    pattern = r"([A-Z][a-zA-Zçéàèíóõâêã]+\s+[A-Z][a-zA-Zçéàèíóõâêã]+.*?)\s([\w\.\s]+?)\s([\w\.-]+@novaims\.unl\.pt)"
    matches = re.findall(pattern, text)
    
    grouped_text = ""
    chunks = []
    current_chunk_token_count = 0  # Keep track of the current chunk's token count

    for match in matches:
        name, title, email = match
        professor_info = f"{name}\n{title}\n{email}\n\n"
        
        # Estimate the number of tokens in the new professor info (rough estimate by splitting on space)
        token_count = len(professor_info.split())

        # If adding this professor exceeds the max token size, start a new chunk
        if current_chunk_token_count + token_count > max_token_size:
            if grouped_text:
                # Add the current chunk as a new document and reset
                chunks.append(Document(page_content=grouped_text, metadata=metadata or {}))
                grouped_text = ""  # Reset for the next group
                current_chunk_token_count = 0  # Reset token count for the new chunk
        
        # Add the current professor to the group
        grouped_text += professor_info
        current_chunk_token_count += token_count

    # Add the last chunk if there's any remaining text
    if grouped_text:
        chunks.append(Document(page_content=grouped_text, metadata=metadata or {}))

    return chunks

In [9]:
# Extract the text content from the Document
staff_text = all_documents[6].page_content

# Pass the extracted text to the split function
staff_chunks = split_teaching_staff_grouped(staff_text, metadata={"source": "teaching_staff"})

# Print the number of chunks and the content of the first chunk
print(len(staff_chunks))
print(staff_chunks[0].page_content)

2
Teaching Staff en Programs Bachelor's
Degrees Data Science Teaching Staff Américo Rio Invited Assistant Professor
americo.rio@novaims.unl.pt

Ana Cristina
Costa Associate Professor
cristina@novaims.unl.pt

Artur Varanda
Adjunct Lecturer
avaranda@novaims.unl.pt

Augusto Santos
Assistant Professor
ajrsantos@novaims.unl.pt

Bruno Damásio
Assistant Professor
bdamasio@novaims.unl.pt

Carina Albuquerque
Assistant Professor
calbuquerque@novaims.unl.pt

Carolina Maria
Shaul Adjunct Lecturer
cshaul@novaims.unl.pt

Carolina Santos
Maximiano Adjunct Lecturer
cmaximiano@novaims.unl.pt

Carolina Vasconcelos
Invited Teaching Assistant
cvasconcelos@novaims.unl.pt

Catarina Neves
Assistant Professor
cneves@novaims.unl.pt

Catarina Palha
Invited Teaching Assistant
cpalha@novaims.unl.pt

Dhruv Akshay
Pandit Invited Teaching Assistant
dpandit@novaims.unl.pt

Diogo Rasteiro
Research Assistant
drasteiro@novaims.unl.pt

Fernando Bação
Full Professor
bacao@novaims.unl.pt

Filipe Marques
Professor of the Pr

# Costum chunking for Study plan

- Chunking by year and semestres and having a argument to set the total number of tokens to have in each document. Optional Chunk_overlap 

In [ ]:
import re
from typing import List, Dict
from langchain_core.documents import Document
from transformers import AutoTokenizer, PreTrainedTokenizerFast

'''
# Without overlap (default)
docs = chunk_all_documents(my_texts_dict, max_tokens=512)

# With 20-token overlap between chunks
docs = chunk_all_documents(my_texts_dict, max_tokens=512, chunk_overlap=True, overlap_size=20)
'''

# Load tokenizer safely
try:
    tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
except Exception as e:
    raise RuntimeError(f"❌ Failed to load tokenizer: {e}")

def count_tokens(text: str) -> int:
    try:
        return len(tokenizer.encode(text, truncation=False))
    except Exception as e:
        print(f"⚠️ Token counting failed for text: {text[:50]}... \nError: {e}")
        return float("inf")

def split_to_token_limit(text: str, max_tokens: int, overlap: bool = False, overlap_size: int = 50) -> List[str]:
    tokens = tokenizer.encode(text, truncation=False)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True).strip()
        chunks.append(chunk_text)
        start = end - overlap_size if overlap else end
    return chunks

def smart_chunk_by_semester(text: str, max_tokens: int = 512, chunk_overlap: bool = False, overlap_size: int = 50) -> List[str]:
    if not isinstance(text, str) or not text.strip():
        return []

    pattern = re.compile(r"(\d+\s?(?:st|nd|rd|th) year - (?:Fall|Spring) Semester)", re.IGNORECASE)
    matches = list(pattern.finditer(text))

    if not matches:
        print("⚠️ No semester headers found — chunking whole text instead.")
        return split_to_token_limit(text, max_tokens, overlap=chunk_overlap, overlap_size=overlap_size)

    chunks = []
    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        chunk_text = text[start:end].strip()

        if count_tokens(chunk_text) <= max_tokens:
            chunks.append(chunk_text)
        else:
            chunks.extend(split_to_token_limit(chunk_text, max_tokens, overlap=chunk_overlap, overlap_size=overlap_size))

    return chunks

def chunk_all_documents(
    doc_dict: Dict[str, str],
    max_tokens: int = 512,
    chunk_overlap: bool = False,
    overlap_size: int = 50
) -> List[Document]:
    
    all_docs = []

    if not isinstance(doc_dict, dict):
        raise ValueError("❌ Input must be a dictionary with document names and text.")

    for name, text in doc_dict.items():
        if not isinstance(text, str) or not text.strip():
            print(f"⚠️ Skipping empty or invalid text for: {name}")
            continue
        try:
            chunks = smart_chunk_by_semester(
                text,
                max_tokens=max_tokens,
                chunk_overlap=chunk_overlap,
                overlap_size=overlap_size
            )
            for chunk in chunks:
                all_docs.append(Document(page_content=chunk, metadata={"source": name}))
        except Exception as e:
            print(f"❌ Error chunking document '{name}': {e}")
    
    return all_docs




In [15]:
from pathlib import Path
import pickle

with open("../../data/Preprocessing_text/bachelors_data/dict_studyplan_cleaned.pkl", "rb") as f:
    studyplan_dict = pickle.load(f)

docs = chunk_all_documents(studyplan_dict, max_tokens=512)

print(f"Total docs: {len(docs)}")
print(docs[0].metadata)

Total docs: 18
{'source': 'data-science_program_extracted_text.txt'}


In [16]:
docs

[Document(metadata={'source': 'data-science_program_extracted_text.txt'}, page_content="1 st year - Fall Semester Course Units Duration Type Contact Hours Total Workload ECTS Computers' Architecture Semester Mandatory TP - 45 | OT - 5 112 4 Foundational aspects of data science Semester Mandatory TP - 45 | OT - 5 112 4 Information Systems Semester Mandatory TP - 64,5 | S - 3 | OT - 7,5 168 6 Introduction to Programming Semester Mandatory TP - 67,5 | OT - 7,5 196 7 Linear Algebra Semester Mandatory T - 22,5 | PL - 22,5 | OT - 7,5 112 4 Show subtitles T – Theoretical Teaching; TP – Theoretical/ Practical Teaching; TC – Field Work; PL – Theoretical/ Practical Teaching; S – Seminar; OT – Tutor Group; ** – Course Unit available in both semesters;"),
 Document(metadata={'source': 'data-science_program_extracted_text.txt'}, page_content='1 st year - Spring Semester Course Units Duration Type Contact Hours Total Workload ECTS Algorithms and Data Structures Semester Mandatory TP - 67,5 | OT - 7,

In [17]:
for i, doc in enumerate(docs):
    num_tokens = count_tokens(doc.page_content)
    print(f"Chunk {i+1}: {num_tokens} tokens")
    if num_tokens > 512:
        print("❗️WARNING: Token limit exceeded")

Chunk 1: 164 tokens
Chunk 2: 218 tokens
Chunk 3: 177 tokens
Chunk 4: 159 tokens
Chunk 5: 177 tokens
Chunk 6: 323 tokens
Chunk 7: 182 tokens
Chunk 8: 186 tokens
Chunk 9: 177 tokens
Chunk 10: 173 tokens
Chunk 11: 194 tokens
Chunk 12: 485 tokens
Chunk 13: 182 tokens
Chunk 14: 184 tokens
Chunk 15: 165 tokens
Chunk 16: 207 tokens
Chunk 17: 186 tokens
Chunk 18: 460 tokens


In [13]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Utilizador\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [15]:
import pickle
import re
import tiktoken
from nltk import sent_tokenize
from typing import List, Dict

# === Helpers ===

def num_tokens_from_string(string: str, model_name: str = "gpt-3.5-turbo") -> int:
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(string))

def is_potential_section_header(line: str) -> bool:
    line = line.strip()
    if not line:
        return False
    return (
        len(line) < 80 and (
            line.isupper() or
            line.istitle() or
            bool(re.match(r"^\d+[\).\s]", line))
        )
    )

# === Main Chunking Logic ===

def chunk_dict_sections_with_headings_and_overlap(
    path: str,
    max_tokens: int = 800,
    overlap: int = 100,
    model_name: str = "gpt-3.5-turbo"
) -> Dict[str, List[str]]:
    with open(path, "rb") as f:
        dict_data = pickle.load(f)

    chunked_data = {}

    for course_name, text in dict_data.items():
        lines = text.split("\n")
        sections = []
        current_section = {"header": None, "content": ""}

        for line in lines:
            if is_potential_section_header(line):
                if current_section["header"] or current_section["content"].strip():
                    sections.append(current_section)
                current_section = {"header": line.strip(), "content": ""}
            else:
                current_section["content"] += line.strip() + " "

        if current_section["header"] or current_section["content"].strip():
            sections.append(current_section)

        course_chunks = []

        for section in sections:
            full_text = (section["header"] + "\n" if section["header"] else "") + section["content"]
            sentences = sent_tokenize(full_text)
            buffer = []
            token_count = 0

            i = 0
            while i < len(sentences):
                sentence = sentences[i]
                sentence_tokens = num_tokens_from_string(sentence, model_name)

                if token_count + sentence_tokens <= max_tokens:
                    buffer.append(sentence)
                    token_count += sentence_tokens
                    i += 1
                else:
                    # Save chunk
                    chunk = " ".join(buffer)
                    course_chunks.append(chunk)
                    # Overlap
                    overlap_buffer = []
                    overlap_token_count = 0
                    for s in reversed(buffer):
                        s_tokens = num_tokens_from_string(s, model_name)
                        if overlap_token_count + s_tokens <= overlap:
                            overlap_buffer.insert(0, s)
                            overlap_token_count += s_tokens
                        else:
                            break
                    buffer = overlap_buffer
                    token_count = overlap_token_count

            if buffer:
                course_chunks.append(" ".join(buffer))

        chunked_data[course_name] = course_chunks

    return chunked_data



In [20]:
path = "../../data/Preprocessing_text/bachelors_data/dict_maininfo_cleaned.pkl"
chunked = chunk_dict_sections_with_headings_and_overlap(path, max_tokens=800, overlap=100)

# Iterate through each course and print each chunk with the number of tokens
for course_name, chunks in chunked.items():
    print(f"Course: {course_name}")
    for idx, chunk in enumerate(chunks):
        num_tokens = len(chunk.split())  # Count tokens by splitting on spaces (a simple tokenization method)
        print(f"\n--- Chunk {idx + 1} ---")
        print(f"Number of tokens: {num_tokens}")
        print(chunk)
        print("\n--- End of Chunk ---\n")


Course: data-science_main_course_extracted_text.txt

--- Chunk 1 ---
Number of tokens: 642
Text from https://www.novaims.unl.pt/en///bachelor-s-degrees/data-science/: Data Science Degree in Data Science en Bachelor's Degrees Data Science In the Bachelor´s Degree in Data Science, students learn the most modern techniques of artificial intelligence and machine learning to analyze large volumes of data (Big Data). They will become true data scientists - considered the sexiest profession of the 21 st century by the Harvard Business Review. The main objective of this course is to train future professionals capable of understanding, developing and using models, algorithms and the most advanced techniques in data science, to analyze and extract knowledge from Big Data. The 3 rd phase of applications under the International Student Statute for the 2025/26 academic year are open from February 26 th to March 27 th , 2025. Duration 3 years (6 semesters) Timetable Daytime Start September 2025 Care